# 1-Upload_Imagery
This notebook is used for uploading drone imagery to the DIST calval database.

In [1]:
import geopandas as gpd
import pandas as pd
import boto3
import os
from datetime import datetime
import sys
sys.path.insert(0, './tools/')
from uploadImage import uploadImage
from pathlib import Path
import rasterio
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds

YOU NEED TO FILL THESE OUT!

In [ ]:
#Essential metadata fields
image_name = '' 
date_acquired = ''
instrument = ''
observation_type = ''
dist_product_id = ''
notes = ''

#Local location of image to be uploaded
path_to_image_file = ''
path_to_derived_file = ''

fields = [image_name, date_acquired, instrument, observation_type, dist_product_id, path_to_image_file]
if any([not f for f in fields]):
    raise ValueError

### AWS Credentials
In order to upload imagery to bucket, JPL RSA access and OPERA Calval AWS credenitals are needed

In [ ]:
bucket_name = 'opera-calval-database-dswx'
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

# Check that Imagery Exists at Specified Path

In [ ]:
#Local directory where imagery file(s) are located
uploadPath = Path(path_to_image_file).absolute()
assert((uploadPath).exists())

## Get the polygon bounds of imagery

In [ ]:
# Classified Image read and geometry

with rasterio.open(uploadPath) as ds:
    bounds = ds.bounds
    crs_utm = ds.crs
    
bounds_4326 = transform_bounds(crs_utm, CRS.from_epsg(4326), *bounds)
geometry = box(*bounds_4326)
geometry

### Enter the required file locations and metadata fields
To upload the imagery, we need to specify its location on the local computer (and the location of auxilary files). We also need to fill in some metadata fields. Both file paths and metadata are specified as dictionaries

In [ ]:
filePaths = {
    'image' : str(uploadPath),
    'derived' : str(Path(path_to_derived_file).absolute())
}
filePaths

In [ ]:
metaData = {
    'image_name' : image_name, 
    'date_acquired' : date_acquired,
    'instrument' : instrument,
    'observation_type' : observation_type,
    'dist_product_id' : dist_product_id,
    'notes' : notes,
    'public':True, #bool
    'geometry': geometry, #shapely geometry
}
metaData

### Stage the image
We use a pre-defined function to upload files and metadata to the staging area. This function takes the file paths and metadata dictionaries, as well as the AWS session object as inputs

    

In [ ]:
uploadImage(filePaths,metaData,session)